## Импорты

In [1]:
import json

from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
import RAKE
import nltk
from nltk.corpus import stopwords

from summa import keywords

import yake

m = MorphAnalyzer()
stop = stopwords.words('russian')

In [2]:
with open('data.json', encoding='utf-8') as json_file:
    data = json.load(json_file)

## Функции нормализации и нормализации-фильтрации текста

In [3]:
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)


def normalize_and_filter(text, filt):
    f = filt.split('+')
    lemmas = []
    coloc = ''
    for i in range(len(simple_word_tokenize(text))-len(f)):
        for j in range(len(f)):
            if m.parse(simple_word_tokenize(text)[i+j])[0].tag.POS == f[j]:
                coloc += ' '+ m.parse(simple_word_tokenize(text)[i+j])[0].normal_form
            else:
                coloc = ''
                break
        if coloc:
            lemmas.append(m.parse(coloc)[0].normal_form)
    return ''.join(lemmas)

# RAKE

In [4]:
rake = RAKE.Rake(stop)

for text in range(len(data)):
    raked = rake.run(normalize_text(data[text]['text']), maxWords=3, minFrequency=2)
    print(str(text + 1) + '-й текст:')
    for i in range(len(raked)):
        print(raked[i][0], end='; ')
    print('\n')

1-й текст:
число западно-европейский; xvi-xvii век; второй половина xvii; европейский россия; северо-западный; конец xvi; культура манси; шёлковый ткань; российский производство; хлопчатобумажный ткань; юго-западный; ирбитский ярмарка; начало xviii; по-видимому; шерстяной ткань; развитие торговля; средний азия; шёлковый; халат; ткань; число; xviii; китайский; сшить; сукно; xvii; хлопчатобумажный; южный; восточный; привозиться; шерстяной; проживать; связать; различный; сибирь; сосьва; среднеазиатский; хант; ляпин; связь; период; сарафан; это; кама; образ; берёзовый; н; чердынь; верхотурий; путь; др; печора; чусовая; бахрушин; соколов; урал; приток; кроме; мангазя; отличие; китай; продаваться; атлас; пр; татарин; далее; ситец; сатина; гемуесть; посолдино; рис; близкий; очевидно; 2007; 196; 1990; 2000; 1975; 1967; 1; 2; 3; 1999; 1997; 

2-й текст:
императорский публичный библиотека; публичный библиотека; время командировка; передать п; собрание п; начало xix; фонд эрмитаж; материал; колле

## Rake на отфильтрованном тексте NOUN+NOUN

In [5]:
for text in range(len(data)):
    raked = rake.run(normalize_and_filter(data[text]['text'], 'NOUN+NOUN'), maxWords=3)
    print(str(text + 1) + '-й текст:')
    for i in range(len(raked)):
        print(raked[i][0], end='; ')
    print('\n')

1-й текст:


2-й текст:


3-й текст:


4-й текст:




Rake не смог найти ключевые слова в данном подкорпусе

## Rake на отфильтрованном тексте ADJF+NOUN

In [6]:
for text in range(len(data)):
    raked = rake.run(normalize_and_filter(data[text]['text'], 'ADJF+NOUN'), maxWords=3)
    print(str(text + 1) + '-й текст:')
    for i in range(len(raked)):
        print(raked[i][0], end='; ')
    print('\n')

1-й текст:
образ торгово-обменный отношение; путь русский городка; образ северный манси; ткань долгий время; ткань иноэтничный происхождение; ткань традиционный одежда; образ импортный ткань; образ; ткань; 

2-й текст:
образ музейный деть; место; номер; факт; 

3-й текст:
образ морально-психологический состояние; 

4-й текст:
кровля археологический культура; работа растительный отпечатка; 



# TextRank

In [7]:
for text in range(len(data)):
    k_words = keywords.keywords(normalize_text(data[text]['text']), language='russian', additional_stopwords=stop, scores=True)
    print(str(text + 1) + '-й текст:')
    for i in range(len(k_words)):
        print(k_words[i][0], end='; ')
    print('\n')

1-й текст:
манси ткань; торговый; халат; товар; который; западный; западно; торгово отношение; одежда; шёлковый; иметь; иметься; русский; среда; хлопчатобумажный; вилковый; сибирь; числиться среди; рынок; человек; тобольск; тобольский; берёзовый; восточный; xvii; xviii; xvi; зенденной; период основный часть население; китайский; число свой; ряд; рядом; сарафан; бухарский дорога; шерстяной; путь; путём; поступление; верхотурий; век; торговля; шёлк; значение; особый роль; чердынь; сукно; зендени чёрный; проживание; это район; северный; европейский; ирбитский; занимать; первый; анализ; близкий; сибирский; среднеазиатский; происхождение; начало; выходить; розово букет; территория россия; ярмак; мансийский; место; ввоз; ввозить; ввозиться; осуществляться; зырянский; ранний; рано; ранее; южный; река вишера; усть; российский; основное; группа; также; производство; выткать; рис; считать; б; стиль; ярмарочный; напрямую; сшить; исследование; привозиться; привозить; время; культовый; изготовление

## TextRank на отфильтрованном тексте NOUN+NOUN

In [8]:
for text in range(len(data)):
    k_words = keywords.keywords(normalize_and_filter(data[text]['text'], 'NOUN+NOUN'), language='russian', additional_stopwords=stop, scores=True)
    print(str(text + 1) + '-й текст:')
    for i in range(len(k_words)):
        print(k_words[i][0], end='; ')
    print('\n')

1-й текст:
манси; халат; ткань; торговля; поступление; шёлк; территория россия страна; одежда; ха; рынок тобольск; район; проживание; часть население; ярмак; стиль; приуралье; течение; 

2-й текст:
п; коллекция; собрание; материал; вещь; коллекционер; музей; интерес; комплектование; время командировка; круг; часть; список предмет; основа; эрмитаж; фрол; фролов; хранение; период; река; деятельность; коллекционирование собирание; памятник; библиотека; начало; 

3-й текст:
решение состав армия; сила рф; войско; задача характеристика; года; безопасность государство; страна; оборона; войный; степень; 

4-й текст:
федоровский; жилище; хозяйство эпоха бронза группа; раскопка; культура; кость; поселение; изучение; материал; ольховка комплекс; результат; животный; отпечаток; характеристика; скотоводство животное; керамика; растение керамик; пол постройка; ареал; полый; 



## TextRank на отфильтрованном тексте ADJF+NOUN

In [9]:
for text in range(len(data)):
    k_words = keywords.keywords(normalize_and_filter(data[text]['text'], 'ADJF+NOUN'), language='russian', additional_stopwords=stop, scores=True)
    print(str(text + 1) + '-й текст:')
    for i in range(len(k_words)):
        print(k_words[i][0], end='; ')
    print('\n')

1-й текст:
торгово; ткань культовый; человек торговый; дорога; западно; восточный товар; бухарский дорогий; мансийский население; хлопчатобумажный; русский; шёлковый; связь западный; место; европейский; свой; россия; тобольский рынка; основный часть северный; одежда; местный производство; группа; первый; розово; ирбитский ярмарка; особый; центр; зеленовато; зеленоватый; березовский; различный; половина; значение; северо сибирь; манси; отечественный; фон; китайский; старый; отдельный; золотисто; использование; 

2-й текст:
часть; археологический предмет; материал; частный коллекция; книга частый командировка; публичный библиотека; горный округ; деятельность первый; музей; рукописный; этнографический; медный; образ; исследователь научный; собрание; язык; собственный; место; искусство древний; второй; свой; музейный; старинный; китайский; завод; личный; состав; губерния; бронзовый; гость; сибирский древность; 

3-й текст:
морально состояние; моральный дух; психологический; воинский; военн

# YAKE

In [10]:
kw_extractor = yake.KeywordExtractor()
custom_kw_extractor = yake.KeywordExtractor(lan='ru', n=3, dedupLim=0.9, top=20, features=None)

for text in range(len(data)):
    k_words = custom_kw_extractor.extract_keywords(normalize_text(data[text]['text']))
    print(str(text + 1) + '-й текст:')
    for i in range(len(k_words)):
        print(k_words[i][0], end='; ')
    print('\n')

1-й текст:
хлопчатобумажный ткань китайский; хлопчатобумажный ткань; шёлковый ткань; культура манси ткань; восточный хлопчатобумажный ткань; ткань; манси ткань являться; образ импортный ткань; шёлковый ткань западно-европейский; хлопчатобумажный ткань импортироваться; российский шёлковый ткань; импортный ткань войти; восточный товар ткань; целое шёлковый ткань; занимать хлопчатобумажный ткань; шёлковый ткань входить; традиционный одежда манси; ткань являться традиционный; доставка шёлковый ткань; фактура шёлковый ткань; 

2-й текст:
колывано-воскресенский горный округ; начальник колывано-воскресенский горный; барнаульский горный музей; учиться горный училище; окончание горный училище; фролов; коллекция; собрание музей петербург; фролов проявить интерес; особенность комплектование коллекция; музей различный профиль; материал этнографический коллекция; императорский публичный библиотека; коллекция пётр козьмич; организация барнаульский горный; сборник неоднократно освещаться; неоднократн

## Yake на отфильтрованном тексте NOUN+NOUN

In [11]:
for text in range(len(data)):
    k_words = custom_kw_extractor.extract_keywords(normalize_and_filter(data[text]['text'], 'NOUN+NOUN'))
    print(str(text + 1) + '-й текст:')
    for i in range(len(k_words)):
        print(k_words[i][0], end='; ')
    print('\n')

1-й текст:
литература название название; название название халат; название халат халат; совпадение период период; направление поступление направление; период период выработка; поступление направление поступление; направление поступление поступление; халат халат дух-покровитель; название литература название; название халат название; литература название литература; период совпадение период; название халат литература; халат литература название; шёлк литература название; дух-покровитель литература название; совпадение период совпадение; халат название халат; анализ направление поступление; 

2-й текст:
начальник должность начальник; должность начальник начальник; комплектование комплектование коллекция; должность начальник должность; начальник начальник колывано-воскресенский; собрание часть собрание; часть собрание собрание; особенность комплектование комплектование; часть собрание часть; собрание состав собрание; комплектование коллекция коллекция; коллекция коллекция пётр; интерес интер

## Yake на отфильтрованном тексте ADJF+NOUN

In [12]:
for text in range(len(data)):
    k_words = custom_kw_extractor.extract_keywords(normalize_and_filter(data[text]['text'], 'ADJF+NOUN'))
    print(str(text + 1) + '-й текст:')
    for i in range(len(k_words)):
        print(k_words[i][0], end='; ')
    print('\n')

1-й текст:
шёлковый ткань крупный; хлопчатобумажный ткань среднеазиатский; ткань восточный товар; северный сосьва северный; восточный товар европейский; товар европейский россия; ткань хлопчатобумажный ткань; ткань крупный рубчик; ткань шёлковый ткань; шёлковый ткань тобольский; хлопчатобумажный ткань восточный; россия хлопчатобумажный ткань; ткань тобольский рынка; шёлковый ткань восточный; производство хлопчатобумажный ткань; хлопчатобумажный ткань бухарский; одежда хлопчатобумажный ткань; производство шёлковый ткань; российский ткань хлопчатобумажный; дорогий хлопчатобумажный ткань; 

2-й текст:
материал публичный библиотека; горный училище горный; материал количественный состав; округ публичный библиотека; публичный библиотека собирательский; культура публичный библиотека; публичный библиотека горный; интеллигенция публичный библиотека; публичный библиотека скифский; публичный библиотека литой; горный округ публичный; публичный библиотека рукописный; публичный библиотека этнографич